In [ ]:
import pandas as pd
data = pd.read_csv("/home/pradnya/monash/5147/viz-2/final_data.csv", sep='\t')

In [ ]:
data_hail = data[data['Trip_type '] == 1]

In [ ]:
data_hail = data[(data['lpep_pickup_datetime']>'10/01/2016 00:00:00 AM') & (data['lpep_pickup_datetime']<='12/31/2016 23:59:59 PM')]
len(data_hail)

In [ ]:
data_hail['lpep_pickup_datetime'].min()
data_hail.head()

In [ ]:
data_hail = data_hail.drop(columns=['Unnamed: 0','Unnamed: 0.1', 'Store_and_fwd_flag', 'Extra', 'MTA_tax', 'Tolls_amount', 'Ehail_fee', 'improvement_surcharge', 'Total_amount'])

In [ ]:
data_hail = data_hail[ (data_hail['Pickup_longitude'] >= -74.2748) & (data_hail['Pickup_longitude'] <= -73.70741) & \
              (data_hail['Pickup_latitude'] >= 40.45326) & (data_hail['Pickup_latitude'] <= 40.94788) & \
              (data_hail['Dropoff_longitude'] >= -74.2748) & (data_hail['Dropoff_longitude'] <= -73.70741) & \
              (data_hail['Dropoff_latitude'] >= 40.45326) & (data_hail['Dropoff_latitude'] <= 40.94788)]

In [ ]:
data_hail['Pickup_longitude'] = data_hail['Pickup_longitude'].apply(lambda x: round(x, 5))
data_hail['Pickup_latitude'] = data_hail['Pickup_latitude'].apply(lambda x: round(x, 5))
data_hail['Dropoff_longitude'] = data_hail['Dropoff_longitude'].apply(lambda x: round(x, 5))
data_hail['Dropoff_latitude'] = data_hail['Dropoff_latitude'].apply(lambda x: round(x, 5))

In [ ]:
actual_data = data_hail[data_hail['Trip_distance']!=0.0]
out_tdist = data_hail[data_hail['Trip_distance']==0.0]

from math import radians, cos, sin, asin, sqrt

def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 3956 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

out_tdist['Trip_distance']=out_tdist.apply(lambda row:haversine(row['Pickup_longitude'],row['Pickup_latitude'],row['Dropoff_longitude'],row['Dropoff_latitude']),axis=1)
out_tdist[["Pickup_longitude", "Pickup_latitude", "Dropoff_longitude", "Dropoff_latitude", "Trip_distance", "Fare_amount"]]

In [ ]:
data_hail = actual_data.append(out_tdist)
print(len(data_hail))

In [ ]:
import numpy as np

spl = []
spl.append(data_hail.iloc[:1000000])
spl.append(data_hail.iloc[1000000:2000000])
spl.append(data_hail.iloc[2000000:3000000])
spl.append(data_hail.iloc[3000000:])
len(spl)

In [ ]:
lf = spl[3]
actual_data = lf [lf['Fare_amount'] >= 0]
out_fare = lf [lf['Fare_amount'] < 0]
len(out_fare)

In [ ]:
df = pd.DataFrame(columns=list(out_fare.columns.values))

for i in range(len(out_fare)):
    test_df = out_fare.iloc[i]
    tf = data_hail[ (data_hail['Pickup_longitude'] == test_df['Pickup_longitude']) & (data_hail['Pickup_latitude'] == test_df['Pickup_latitude']) \
         & (data_hail['Dropoff_longitude']== test_df['Dropoff_longitude']) & (data_hail['Dropoff_latitude'] == test_df['Dropoff_latitude']) \
         & (data_hail['Trip_distance']== test_df['Trip_distance']) ]
    if len(tf) > 1:
        test_df['Fare_amount']= tf[ tf['Fare_amount'] > 0]['Fare_amount'].mean()
        df = df.append(test_df)

In [ ]:
data1 = actual_data.append(df)
len(data1)

In [ ]:
data_hail = data_hail.append(data1)
len(data_hail)

In [ ]:
data_hail['lpep_pickup_datetime'] = pd.to_datetime(data_hail['lpep_pickup_datetime'],errors="coerce")
data_hail['Lpep_dropoff_datetime'] = pd.to_datetime(data_hail['Lpep_dropoff_datetime'],errors="coerce")

In [ ]:
import calendar

data_hail['pickup_date']= data_hail['lpep_pickup_datetime'].dt.date
data_hail['pickup_day']=data_hail['lpep_pickup_datetime'].apply(lambda x:x.day)
data_hail['pickup_hour']=data_hail['lpep_pickup_datetime'].apply(lambda x:x.hour)
data_hail['pickup_day_of_week']=data_hail['lpep_pickup_datetime'].apply(lambda x:calendar.day_name[x.weekday()])
data_hail['pickup_month']=data_hail['lpep_pickup_datetime'].apply(lambda x:x.month)
data_hail['pickup_year']=data_hail['lpep_pickup_datetime'].apply(lambda x:x.year)

In [ ]:
data_hail.head()

In [ ]:
data_hail.to_csv("hailForStats.csv", sep='\t')